# Pytorch_lightning_audio

In [87]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from torchvision import transforms
from torch.nn.utils.rnn import pack_sequence

In [6]:
from tqdm.notebook import tqdm 
import numpy as np

# Datasets読み込み

In [7]:
datasets_dir = "/disk107/yamamoto/datasets.npz"
datasets = np.load(datasets_dir)
data = datasets['arr_0']
label = datasets['arr_1']

In [88]:
practice_d = data
practice_l = label
print(practice_d[1,:,:].shape)

print("Convert RNN Style")
for l in tqdm(range(practice_d.shape[0])):
    pack = [torch.tensor(practice_d[l,:,i]) for i in range(practice_d.shape[2])]
    rnn_datapack = pack_sequence(pack)
    
for l in tqdm(range(practice_l.shape[0])):
    pack = [torch.tensor(practice_l[l,:,i]) for i in range(practice_l.shape[2])]
    rnn_labelpack = pack_sequence(pack)
    
# # 1次元配列変換
# _d = np.array([practice_d[i].reshape(-1,) for i in tqdm(range(len(practice_d)))])
# _l = np.array([practice_l[i].reshape(-1,) for i in tqdm(range(len(practice_l)))])

transform = transforms.Compose([
    transforms.ToTensor(), 
#     transforms.Lambda(lambda x: x.view(-1))
])

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data, label, transform=None):
        self.transform = transform
        self.data = data
        self.data_num = len(data)
        self.label = label

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        if self.transform:
          out_data = self.transform(self.data)[0][idx]
          out_label = self.label[idx]
        else:
          out_data = self.data[idx]
          out_label =  self.label[idx]

        return out_data, out_label

(127, 512)


In [9]:
mydataset = MyDataset(rnn_datapack, rnn_labelpack, transform = None)

# ネットワークの定義
**sizeの設定気をつけて！**

In [98]:
input_size = 512
hidden_size = 512
output_size = 512

## LSTM
2 rayers

In [99]:
#lstm
class LSTM(pl.LightningDataModule):
    def __init__(self):
        super(LSTM, self).__init__()
        self.LSTM = nn.Sequential(
            nn.LSTM(input_size = input_size,hidden_size = hidden_size),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        # Lightningではforwardは予測/推論のアクションを定義します。
        embedding = self.LSTM(x)
        return embedding
    
    def configure_optimizers(self):# Optimaizerの設定
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):# 完全なトレーニングループ
        criterion = nn.L1Loss()
        # training_stepでは訓練ループを定義していて、これはforwardとは独立しています
        x, y = batch
        z = self.LSTM(x)  # forwardを呼び出す self(x) でもよい
        loss = criterion(z, y)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):#完全な検証ループ
        criterion = nn.L1Loss()
        # 検証ステップ
        x, y = val_batch
        z = self.LSTM(x)
        loss = criterion(z, y)
        self.log("val_loss", loss)
        return loss

In [100]:
# train : val = 80% : 20%
n_train = int(len(mydataset) * 0.8)
n_val = len(mydataset) - n_train
train, val = random_split(mydataset, [n_train, n_val])

print(len(train),"train, ",len(val),"val")

12480 train,  3120 val


In [101]:
#モデルの呼び出しと初期化
model = LSTM()# 学習モデルのインスタンス化
#トレーニング開始
trainer = pl.Trainer(max_epochs=10, gpus=4)# 学習用のインスタンス化と学習の設定

trainer.fit(model, 
            DataLoader(train, batch_size = 10), 
            DataLoader(val, batch_size = 10))# 学習ループ実行
# batchサイズはdata数に対して割り切れる数を渡す
# ここでdataLoaderした訓練データと検証データをわたす。

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.